In [13]:
from astropy.io import fits
import numpy as np
import glob

In [22]:
!ls Wolf1346/bias.00??.fits > bias.list
biasfiles = [line.rstrip('\n') for line in open('bias.list')]

### MASTER BIAS

In [23]:
data = []
for file in biasfiles:
    data.append(fits.getdata(file))

In [27]:
avgBias = np.median(data, axis=0)

header = fits.getheader(biasfiles[0])

header['HISTORY'] = 'Median combined'
fits.writeto('Wolf1346/bias.fits', avgBias, header, overwrite=True)

### SUBTRACT BIAS

In [28]:
!ls Wolf1346/flat.00??.fits > before.list

In [29]:
!ls Wolf1346/wolf1346_.000?.fits >> before.list

In [30]:
!ls Wolf1346/wolf1346_m.0001.fits >> before.list

In [31]:
datafilesin = [line.rstrip('\n') for line in open('before.list')]
datafilesout = [line.rstrip('\n') for line in open('refined.list')]

n = len(datafilesin)
for i in np.arange(n):
    data, header = fits.getdata(datafilesin[i], header=True)
    dataout = data - avgBias
    fits.writeto(datafilesout[i], dataout, overwrite=True)

### OVERSCAN REMOVAL

In [34]:
data = fits.getdata("Wolf1346/flatOR.0005.fits")

In [35]:
data.max()

45393.0

In [36]:
data.min()

-65046.0

In [37]:
np.shape([[1,2],[3,5],[4,5]])

(3, 2)

In [38]:
np.shape(data[:,1027:1077])

(2050, 50)

In [39]:
overscan_removed1 = np.delete(data, np.s_[1026:1078], axis=1)

In [40]:
overscan_removed2 = np.delete(overscan_removed1, np.s_[1024:1028], axis=0)

In [41]:
print(np.shape(overscan_removed1), np.shape(overscan_removed2))

(2050, 2050) (2046, 2050)


In [42]:
fits.writeto('Wolf1346/flat.0005d.fits', overscan_removed2, overwrite=True)

In [44]:
readin = [line.rstrip('\n') for line in open('refined.list')]
readout = [line.rstrip('\n') for line in open('overscan.list')]
n = len(readin)

In [45]:
for i in np.arange(n):
    data = fits.getdata(readin[i])
    data1 = np.delete(data, np.s_[1026:1078], axis=1)
    data2 = np.delete(data1, np.s_[1024:1028], axis=0)
    fits.writeto(readout[i], data2, overwrite=True)